# Unit 5: Advanced NumPy

We already encountered NumPy arrays and their basic usage
in unit 1. In this unit, we will take a more in-depth look
at NumPy arrays.

## Why NumPy arrays?

Why don't we just stick with built-in types such as Python
lists to store and process data? It turns out that while the built-in objects
are quite flexible, this flexibility comes at the cost
of performance:
-   `list` objects can store arbitrary data types, and
    the data type of any item can change:
    ```
    items = ['foo']
    items[0] = 1.0      # item was a string, now it's a float!
    ```
-   There is no guarantee where in memory the data will be stored.
    In fact, two consecutive items could be very "far"
    from each other in memory, which imposes a performance
    penalty.
-   Even primitive data types such as `int` and `float` are
    not such "raw" data, but full-fledged objects.
    That, again, is bad for performance.

On the other hand, the approach taken by NumPy is to
store and process data in a way very similuar to low-level
languages such as _C_ and *Fortran*.
This means that
-   Arrays contain a *homogenous* data type. *All* elements are either
    64-bit integers (`np.int64`), 64-bit floating-point numbers
    (`np.float64`), or some other of the many data types
    supported by NumPy.

    It is technically possible to get around this by specifying
    an array's data type (`dtype`) to be `object`, which is the most
    generic Python data type. However, we would never want to
    do this for numerical computations.
-   NumPy arrays are usually *contiguous* in memory. This means
    that adjacent array elements are actually guaranteed to be
    next to each other in memory, which allows for much more efficient
    computations.
-   NumPy array support numerous operations used in
    scientific computing. For example, with a NumPy array we can write
    ```
    x = np.array([1, 2, 3])
    y = x + 1       # We would expect this to work
    ```
    With lists, however, we cannot:
    ```
    x = [1, 2, 3]
    y = x + 1       # Does not work!
    ```
    Lists don't implement an addition operator that accepts
    integer arguments, so this code triggers an error.

You can see performance uplift provided by NumPy arrays in simple
example:

In [11]:
# Create list 0, 1, 2, ..., 999
lst = list(i for i in range(1000))

# Compute squares, time how long it takes
%timeit [i**2 for i in lst]

204 µs ± 9.84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
# Repeat using NumPy arrays
import numpy as np
arr = np.arange(1000)

%timeit a**2

185 ns ± 10 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


-   On my machine (not the fastest laptop), squaring 1000 elements of a `list` takes
    197 microseconds, while the equivalent array-based operation takes only
    178 nanoseconds. NumPy is therefore approximately 1000 times faster!
-   Also, as mentioned above, NumPy supports squaring an array directly, while
    we have to manually loop through the `list` and square each element
    individually.

*Note:* `%timeit` is a so-called magic command that only
works in notebooks, but not in regular Python files.
[[See documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html)]

***

## Creating arrays

We have already encountered some of the most frequently
used array creation routines:
-   `np.array()` creates an array from a given argument, which can be
    -   a scalar
    -   a collection such as a list or tuple
    -   some other iterable object, eg. something created by `range()`
-   `np.empty()` allocates memory for a given array shape, but does not
    overwrite it with initial values.
-   `np.zeros()` creates an array of a given shape and initializes it
    to zeros.
-   `np.ones()` creates an array of a given shape and initializes it
    to ones.
-   `np.arange(start,stop,step)` creates an array with evenly spaced
    elements over the range $[start,stop)$.
    -   `start` and `step` can be omitted and then default to `start=0` and `step=1`.
    -   Note that `stop` is not included!
-   `np.linspace(start,stop,num)` returns a vector of `num` elements
    which are evelny spaced over the interval $[start,stop]$.
-   `np.identity(n)` returns the identity matrix of a size $n \times n$.
-   `np.eye()` is a more flexible variant of`identity()` that can,
    for example, also create non-squared matrices.

There are many more array creation functions for more exotic use-cases,
see the NumPy  [documentation](https://numpy.org/doc/stable/reference/routines.array-creation.html)
for details.

Some examples:

In [19]:
import numpy as np

# Create array from list
lst = [1, 2, 3]
x = np.array(lst)

array([1., 2., 3.])

In [ ]:
# Create array from tuple
tpl = 1.0, 2.0, 3.0
x = np.array(tpl)
x

In [ ]:
# arange: end point is not included!
x = np.arange(5)
x

In [15]:
# arange: increments can be negative too!
x = np.arange(5, 1, -1)
x

array([5, 4, 3, 2])

In [17]:
# arange also works on floats
x = np.arange(1.0, 3.0, 0.5678)
x

array([1.    , 1.5678, 2.1356, 2.7034])

In [18]:
# linspace DOES include the end point
x = np.linspace(0.0, 1.0, 11)
x

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

***
## Array shape

Many of the array creation take the desired shape of the array
as their first argument.
Array shapes are usually specified as tuples:
-   A vector with5 elements has shape `(5, )`.

    Note the comma `,`: we need to specify a tuple with a single
    element using this comma, since `(5)` is just the integer 5,
    not a tuple.
-   A $2\times2$ matrix has shape `(2, 2)`.
-   A higher-dimensional array has shape `(k, l, m, n, ...)`.
-   A *scalar* NumPy array has shape `()`, an empty tuple.

    While "scalar array" sounds like an oxymoron, it does exist.

We can query the shape of an array using the `shape` attribute,
and the number of dimensions is stored in the `ndim` attribute.

#### Examples

In [22]:
import numpy as np

# Scalar array
x = np.array(0.0)
print('Scalar array with shape={} and ndim={}'.format(x.shape, x.ndim))

Scalar array with shape=() and ndim=0


Note that the a scalar NumPy array is not the same as a Python scalar.
The built-in type `float` has neither a `shape`, nor an `ndim`,
nor any other of the NumPy array attributes.

In [23]:
scalar = 1.0
scalar.shape

AttributeError: 'float' object has no attribute 'shape'

In [24]:
# 1-dimensional array (vector), no initialisation
x = np.empty((5,))
x       # could contain arbitrary garbage

array([4.67337674e-310, 0.00000000e+000, 5.43472210e-322, 4.67337669e-310,
       2.37151510e-322])

An array created with `empty()` will contain arbitrary garbage
since the memory block assigned to the array is not initialised.
The result will potentially differ on each invocation and across
computers.

In [25]:
# 1-dimensional array
x = np.empty(5)     # equivalent to np.empty((5,))

Most function accept an integer value instead of a `tuple`
when creating 1-dimensional arrays, which is interpreted
as the number of elements.

In [26]:
# 3d-array
x = np.ones((1, 2, 3))
x

array([[[1., 1., 1.],
        [1., 1., 1.]]])

Recall from unit 1 that we can use the `reshape()` method
to convert arrays to a different shape:
-   The resulting number of elements must remain unchanged!
-   *One* dimension can be specified using `-1`, which
    will prompt NumPy to compute the implied dimension size
    itself.

In [28]:
x = np.zeros((2, 1, 3))
x = x.reshape((3, -1))      # Infer number of columns
x

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

***
## Advanced indexing

We previously covered single element indexing and slicing,
which worked the same way for both Python `list` and `tuple` objects
as well as NumPy arrays.

NumPy additionally implements more sophisticated indexing mechanisms
which we cover now.
-   You might also want to consult the
NumPy indexing [tutorial](https://numpy.org/doc/stable/user/basics.indexing.html)
and the detailed indexing [reference](https://numpy.org/doc/stable/reference/arrays.indexing.html).

### Boolean or "mask" indexing
We can pass logical arrays as indices.
-   Logical (or boolean) arrays consist of elements that
    can only take on values `True` and `False`
-   We usually don't create logical arrays manually, but apply
    an operation that results in `True`/`False` values,
    such as a comparison.
-   The boolean index array usually has the *same*
    shape as the indexed array.

#### Examples

In [42]:
import numpy as np
vec = np.arange(5)
mask = (vec > 1)        # apply comparison to create boolean array
mask

array([False, False,  True,  True,  True])

In [43]:
vec[mask]               # use mask to retrieve only elements greater than 1

array([2, 3, 4])

We can even apply boolean indexing to multi-dimensional
arrays. The result will be flatted to a 1-dimensional array,
though.

In [44]:
mat = np.arange(6).reshape((2,3))
mat

array([[0, 1, 2],
       [3, 4, 5]])

In [45]:
mat[mat > 1]            # collapses result to 1-d array

array([2, 3, 4, 5])

Logical indexing does **not** work with `tuple` and `list`

In [48]:
tpl = (1, 2, 3)
mask = (True, False, True)
tpl[mask]               # error

TypeError: tuple indices must be integers or slices, not tuple

### Integer index arrays

We can also use index arrays of *integer* type to select
specific elements per axis. They are straightforward to understand
for 1-dimensional arrays, but can get fairly complex with multiple
dimensions:

In [50]:
import numpy as np

data = np.arange(10)
index = [1, 2, 9]       # select second, third and 10th element
data[index]

array([1, 2, 9])

As you see, the index array does not have to be a NumPy array,
but can also be a list (not a tuple, though!).

In general, if we are using an index array to select elements
along an axis of length $n$, the index
-   must only contain integers between 0 and $n-1$, or negative
    integers from $-n$ to $-1$ (which, as usual, count from
    the end of the axis).
-   can be of arbitrary length. We can therefore select the
    same element multiple times.

In [53]:
data = np.arange(5, 10)     # array with 5 elements, [5,...,9]
data

array([5, 6, 7, 8, 9])

In [57]:
index = [0, 1, 1, 2, 2, 3, 3, 4, 4]
data[index]

1

With multi-dimensional data arrays, the same restrictions apply,
and additionally
-   if more than one axis is indexed using index arrays,
    the index arrays have to be of equal length.

In [60]:
data = np.arange(12).reshape((3, 4))
data

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [61]:
index1 = [0, 2]
index2 = [1, 3]
data[index1, index2]

array([ 2, 11])

The code above selects to elements, the first at position
`(0,1)`, the second at position `(2,3)`.

We can combine index arrays on one axis with other indexing methods
used for another axis. For example:

In [62]:
data[index1, 2]     # return elements in 3rd column from rows given in index1

array([ 2, 10])

Using different indexing methods, in particular index arrays,
on higher-dimensional data can quickly become a mess, and you should
be extra careful to see if the results make sense.

***
## Numerical operations

### Elementwise operations
Element-wise operations are performed on each element
individually and leave the resulting array's shape unchanged.

There are three types of such operations:
1.  One operand is an array and one is a scalar.
2.  Both operands are arrays, either of identical shape,
    or broadcastable to an identical shape (we discuss
    broadcasting below)
3.  A function is applied to each array element.

*Case 1:* Array-scalar operations. These intuitively
behave as you would expect:

In [80]:
import numpy as np

x = np.arange(10)
scalar = 1

# The resulting array y has the same shape as x:
y = x + scalar      # addition
y = x - scalar      # subtraction
y = x * scalar      # multiplication
y = x / scalar      # division
y = x // scalar     # division with integer truncation
y = x % scalar      # modulo operator
y = x ** scalar     # power function
y = x == scalar     # comparison: also >, >=, <=, <

Note that unlike in Matlab, the "standard" operators work
element-wise, so `x * y` is **not** matrix multiplication!

*Case 2:* Both operands are arrays of equal shape:

In [81]:
x = np.arange(10)
y = np.arange(10, 20)

# Resulting array z has the same shape as x and y:
z = x + y           # addition
z = x - y           # subtraction
z = x * y           # multiplication
z = x / y           # division
z = x // y          # division with integer truncation
z = x % y           # modulo operator
z = x ** y          # power function
y = x == y          # comparison: also >, >=, <=, <

*Case 3:* Applying element-wise functions
This case covers numerous functions defined by NumPy, such
as
-   `np.sqrt`: square root
-   `np.exp`, `np.log`, `np.log10`: exponential and logarithmic functions
-   `np.sin`, `np.cos`, ...: trigonometric functions

You can find a complete list of mathematical functions in
the NumPy [documentation](https://numpy.org/doc/stable/reference/routines.math.html)
(not all functions listed there operate element-wise!).

In [ ]:
# element-wise functions
x = np.arange(1, 11)
y = np.exp(x)       # apply exponential function
y = np.log(x)       # apply natural logarithm

### Matrix operations

#### Transpose
You can transpose a matrix using the `T` attribute:

In [90]:
mat = np.arange(6).reshape((2,3))
mat

array([[0, 1, 2],
       [3, 4, 5]])

In [94]:
mat.T


array([[[0, 4],
        [2, 6]],

       [[1, 5],
        [3, 7]]])

#### Array multiplication
Matrix multiplication is performed using the `np.dot()`
function ("dot product"). The operands need not be matrixes,
but can be vectors as well, or even high-dimensional arrays
(the result is then not entirely obvious and one should check
the [docs](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)).

Every newer version of Python and NumPy also interprets `@`
as the matrix multiplication operator.

In [83]:
import numpy as np

mat = np.arange(9).reshape((3, 3))
vec = np.arange(3)

# matrix-matrix multiplication
np.dot(mat, mat)    # or: mat @ mat

array([[ 15,  18,  21],
       [ 42,  54,  66],
       [ 69,  90, 111]])

In [84]:
# vector dot product (returns a scalar)
np.dot(vec, vec)    # or: vec @ vec


5

In [86]:
# matrix-vector product (returns vector)
np.dot(mat, vec)    # or: mat @ vec

array([ 5, 14, 23])

We must of course make sure that matrices and vector have
conformable dimensions!

In [87]:
mat = np.arange(6).reshape((2, 3))

In [88]:
np.dot(mat, mat)        # raises error, cannot multiply 2x3 matrix with 2x3 matrix

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)

In [89]:
np.dot(mat, mat.T)      # transpose second operand, works

array([[ 5, 14],
       [14, 50]])

### Reductions

Reductions are operations that reduce the dimensionality of
the data. For example, computing the mean of an array reduces
a collection of data points to a single scalar, its mean.

Basic reduction operations include:
-   `np.sum()`: sum of array elements
-   `np.prod()`: product of array elements
-   `np.amin()`, `np.amax()`: minimum and maximum element
-   `np.argmin()`, `np.argmax()`: location of minimum and maximum element
-   `np.mean()`, `np.average()`: mean of array elements
-   `np.median()`: median of array elements
-   `np.std()`, `np.var()`: standard deviation and variance of array elements
-   `np.percentile()`: percentiles of array elements

Most if not all reductions accept an `axis` argument which
restricts the operation to a specific axis.
-   The resulting array will have one dimension less than the input.
-   If no axis is specified, the operation is applied to the flattened
    array

#### Examples

In [2]:
import numpy as np

# 1-dimensional input data
data = np.linspace(0.0, 1.0, 11)
# Compute mean and std. of input data
m = np.mean(data)
s = np.std(data)
print('Mean: {:.2f}, std. dev.: {:.2f}'.format(m, s))

Mean: 0.50, std. dev.: 0.32


In [5]:
# 2-dimensional input data
data = np.linspace(0.0, 1.0, 21).reshape((3, 7))
data

array([[0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 ],
       [0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65],
       [0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]])

In [6]:
# Compute mean of each row, ie along the column axis
m = np.mean(data, axis=1)
m           # Result is a vector of 3 elements, one for each row

array([0.15, 0.5 , 0.85])

### Broadcasting [advanced]

Element-wise operations in most programming languages require
input arrays to have identical shapes.
NumPy relaxes this constraint and allows us to use arrays
with different shares that can be "broadcast" to identical shapes.

What do we mean by "broadcast"?
-   Imagine we want to add a $2 \times 3$ matrix to a length-2 vector.
-   This operation does not make sense, unless we interpret the (column) vector
    as a $2 \times 1$ matrix, and replicate it 3 times to obtain a
    $2 \times 3$ matrix. This is exactly what NumPy does.

#### Example

In [10]:
import numpy as np

# Create 3x2 matrix
mat = np.arange(6).reshape((2, 3))
mat

array([[0, 1, 2],
       [3, 4, 5]])

In [ ]:
# Create 2-element vector
vec = np.arange(2)
vec

In [ ]:
# Trying to add matrix to vector fails
mat + vec

In [8]:
# However, we can explicitly reshape the vector to a 2x1 column vector
colvec = vec.reshape((-1, 1))
colvec

array([[0],
       [1]])

In [9]:
# Now, broadcasting replicates column vector to match matrix columns
mat + colvec

array([[0, 1, 2],
       [4, 5, 6]])

We do not need to `reshape()` data, but can instead use
a feature of NumPy that allows us to increase the number of
dimensions on the spot:

In [11]:
# use vec[:, None] to append an additional dimension to vec
mat + vec[:, None]

array([[0, 1, 2],
       [4, 5, 6]])

For more examples, see the official NumPy [tutorial](https://numpy.org/doc/stable/user/theory.broadcasting.html)
on broadcasting.


***
## NumPy data types [advanced]

### Default data types
We have already touched upon the numerical data types
used by NumPy. If you do not explicitly request a
data type using the `dtype` keyword argument, NumPy
by default behaves as follows:
1.   The default data type for most array creation routines
    which create arrays of a given shape or size, such as
    `np.empty()`, `np.ones()` and `np.zeros()`,
    is a 64-bit floating-point number (`np.float64`).
2.   Array creation routines that accept numerical input
    data will use the data type of this input data
    to determine the array data type.

    Examples of such functions are `np.arange()` and
    `np.array()`.
3.   Arrays that are implictly created as a result of an
    operation (addition, etc.) are assigned the most suitable
    type to represent the result.

    For example, when adding a floating-point and an
    integer array, the result will be a floating-point array.

#### Examples

*Case 1:* default data type is `np.float64`:

In [ ]:
import numpy as np
x = np.ones(1)      # length-1 vector of ones
x.dtype             # default type: float64

*Case 2:* data type depends on input data:

In [ ]:
# Argument is an integer
x = np.arange(5)
x.dtype     # data type is np.int64 (on most platforms)

In [ ]:
# Argument is a float
x = np.arange(5.0)
x.dtype     # data type is np.float64

*Case 3:* data type determined to accommodate result

In [ ]:
# Add two integer arrays
arr1 = np.arange(3)
arr2 = np.arange(3, 0, -1)      # creates [3, 2, 1]
result = arr1 + arr2
result.dtype        # data type is np.int64

In [ ]:
# Add integer to floating-point array
arr1 = np.arange(3)
arr2 = np.arange(3.0, 0.0, -1.0)    # creates [3.0, 2.0, 1.0]
result = arr1 + arr2
print(result)
result.dtype        # data type is np.float64

Observe that even though the resulting array is `[3.0, 3.0, 3.0]`
and can thus be represented as integers without loss of data,
NumPy only takes into account that one of the operands
is floating-point, and thus the result has to be
of floating-point type!

### Explicit data types

We can almost always explicitly request an array to be
of a particular data type by passing the `dtype` keyword argument.
The most common types are:
-   `np.float64`: a 64-bit floating-point number, also called
    *double precision* in other languages.

    This is the most commonly used floating-point data type. It can
    represent numbers with up to 16 decimal digits, and covers
    a range of approximately $\pm 10^{308}$.

-   `np.int64`: a 64-bit integer which can represent integer
    values on the interval of (approximately) $\pm10^{19}$.

    Unlike with floating-point, the integer representation
    is *exact*, but covers a much smaller range (and, obviously,
    no fractional numbers)

-   `np.float32`, `np.float16`: single-precision and half-precision
    floating-point numbers. These occupy only 32 and 16 bits
    of memory, respectively.

    They thus trade off storage requirements for a loss of
    precision and range.

-   `np.int32`, `np.int16`, `np.int8` represent integers
    using 32, 16 and 8 bits, respectively.

    They require less memory, but can represent only a smaller
    range of integers. For example, `np.int8` can only
    store integer values from $-$128 to 127.

-   NumPy also supports complex numerical types to represent
    imaginary numbers. We will not be using those in this tutorial.

Would we ever want to use anything other than the default
data types, which in most cases are either `np.float64`
and `np.int64`? These, after all, support the largest range
and highest precision. This is true in general, but there are
special cases where other data types need to be used:
1.  *Storage requirements:* if you work with large amounts of
    data, for example arrays with many dimensions, you can run
    out of memory or storage space (when saving results to files).

    In this case, you can store data as `np.float32` instead
    of `np.float64`, which halves the storage requirement.

    Similarly, if you know that your integer data only takes
    on values between $-$128 and 127, you can store them
    as `np.int8` which consumes only an 8-th of the space
    compared to `np.int64`!

2.  *Performance:* Some tasks simply don't require high
    precision or range. For example, some Machine Learning
    tasks can be performed using only 8-bit integers,
    and companies like Google have developed dedicated
    processors to considerably speed up workloads using 8-bit integers.

    Even if you are not using any dedicated CPUs or GPUs,
    data has to be transferred from memory to the processor,
    and this is a major performance bottleneck. The less data needs to
    be transferred, the better!

    In general, this is nothing you need to worry about at this
    point, but might become relevant in the future.

#### Examples

In [ ]:
# Explicitly specify data type
import numpy as np
x = np.ones(1, dtype=np.float16)
x       # prints np.float16

We can use `dtype` to override the data type inferred from
input data:

In [ ]:
lst = [1, 2, 3]
x = np.array(lst)       # given list of integers, creates integer array
x.dtype                 # prints np.int64

In [ ]:
# override inferred data type:
# created floating-point array even if integers were given
x = np.array(lst, dtype=np.float64)
x.dtype                 # prints np.float64

In [ ]:
# override inferred data type:
# created integer array even if floats were given,
# thus truncating input data!
lst = [1.234, 4.567, 6.789]
x = np.array(lst, dtype=np.int)
print(x)                # prints [1, 4, 6]
x.dtype                 # prints np.int64


***

## Copies and views [advanced]

Recall that assignment in Python does *not* create a copy
(unlike in _C_, *Fortran* or *Matlab*):

In [64]:
a = [0, 0, 0]
b = a           # b references the same object as a
b[1] = 1        # modify second element of b (and a!)
a == b          # a and b are still the same

True

NumPy adds another layer to data this sort of data sharing:
whenever you perform an assignment or indexing operation,
NumPy tries hard *not* to copy the underlying data but instead
creates a so-called view point to the same block of memory.
It does this for performance reasons (copying is expensive).

We can illustrate this using array slicing:

In [65]:
import numpy as np

x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [67]:
y = x[3:8]          # Create array that points to elements 4-8 of x
y

array([3, 4, 5, 6, 7])

`x` and `y` are two different Python objects, which we
can verify using the built-in `id()` function:

In [68]:
print(id(x))
print(id(y))
id(x) == id(y)

140136930239552
140136930293600


And yet, the NumPy implementation makes sure that they reference
the same block of memory!

We can see this easily by modifying `y`:

In [70]:
y[:] = 0        # overwrite all elements of y with zeros
y

array([0, 0, 0, 0, 0])

In [71]:
x               # elements of x that are also referenced by y
                # are now also zero!

array([0, 1, 2, 0, 0, 0, 0, 0, 8, 9])

This behaviour is even triggered when `y` references
non-adjacent elements in `x`. For example, we can let `y`
be a view on every *second* element in `x`:

In [73]:
x = np.arange(10)
y = x[::2]      # y now points to every second element of x
y[:] = 0        # overwrite all elements of y with zeros
x               # every second element in x is now zero!

array([0, 1, 0, 3, 0, 5, 0, 7, 0, 9])

As a rule of thumb, NumPy will create a view as opposed to
copying data if
-   An array is created from another array via slicing
    (ie indexing using the `start:stop:step` triplet)

Conversely, a *copy* is created whenever
-   An array is created from another array via boolean
    (mask) indexing.
-   An array is created from another array via integer
    array indexing.

You can always force NumPy to create a copy by calling
`np.copy()`!

#### Examples

In [74]:
# Copies are created with boolean indexing
x = np.arange(10)
mask = (x > 4)      # boolean mask
y = x[mask]         # create y using boolean indexing
y[:] = 0
x                   # x is unmodified

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [75]:
# Copies are creating with integer array indexing
x = np.arange(10)
index = [3, 4, 5]   # List of indices to include in y
y = x[index]
y[:] = 0
x                   # x is unmodified

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [76]:
# Forced copy with slicing
x = np.arange(10)
y = np.copy(x[3:8]) # force copy with np.copy()
y[:] = 0
x                   # x is unmodified

array([3, 4, 5, 6, 7])

As an alterantive to `np.copy()` we can directly call the `copy()`
method of an array:

In [77]:
y = x[3:8].copy()


***

## Array storage order [advanced]
Computer memory is linear, so a multi-dimensional
array is mapped to a one-dimensional block in memory.
This can be done in two ways:
1. NumPy uses the so-called *row-major order* (also called
*C order*, because its the same as in *C* programming language)
2. This is exactly the opposite of Matlab, which uses
  *column-major order* (also called *F order*, because its the same
  as in the *Fortran* programming language)

In [ ]:
mat = np.arange(6).reshape((2,3))
# The matrix mat is stored in memory like this
mat.reshape(-1, order='C')

In [ ]:
# ... and NOT like this
mat.reshape(-1, order='F')

While this is not particularly important initially,
as an advanced user you should remember that you never
want to perform on non-contiguous blocks of memory.
This can have devastating effects on performance!

In [ ]:
# Avoid operations on non-contiguous array sections such as
mat[:, 1]
# Contiguous array sections are fine
mat[1]